# Tracing for Different Types of Runs
LangSmithでは、Run（処理）の種類を分類し、それぞれの役割を指定できます。これは、@traceable デコレーターを使って設定可能です。以下は、サポートされているRunの種類とその概要です。

### Runの種類  
1.LLM（言語モデルの実行）

- 目的: LLM（大規模言語モデル）を呼び出す処理を表します。
- 用途例: テキスト生成、質問への回答、タスクの完了。
- 具体例: OpenAIのGPTモデルにクエリを送信して応答を生成する。 

2.Retriever（情報検索）

- 目的: 知識ベース、データベース、または外部ソースから関連するドキュメントやデータを取得する処理。
- 用途例: 質問への回答に必要なコンテキストや情報を検索する。
- 具体例: ベクトルデータベースをクエリし、特定のトピックに関連する文書を取得する。  

3.Tool（ツールの実行）

- 目的: 外部APIや関数呼び出しを使用して、特定のアクションを実行する処理。
- 用途例: 計算を行う、Webサービスとやり取りする、コマンドを実行する。
- 具体例: 為替レート計算ツールを使用して通貨換算を実行する。  

4.Chain（チェーン）

- 目的: デフォルトのRunタイプで、複数のRunを組み合わせてより大きなプロセスを構築します。
- 用途例: LLM呼び出しと情報検索を連携させたワークフローを作成する。
- 具体例: ユーザー入力を解析し、情報を検索して最終応答を生成するプロセス。  

5.Prompt（プロンプトの作成）  

- 目的: LLMで使用するプロンプトを動的に生成する処理。
- 用途例: ユーザーの入力に応じたプロンプトを作成する。
- 具体例: 自然言語の質問を特定のフォーマットに変換する。  

6.Parser（解析器）

- 目的: 構造化データを抽出する処理。
- 用途例: LLMの出力を解析して、JSONや特定の形式に変換する。
- 具体例: LLMの応答から住所や日付情報を抽出してデータベースに保存する。

In [1]:
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

load_dotenv(dotenv_path="../.env", override=True)

True

## LLM Runs for Chat Models
LangSmithでは、LLMトレースの特別なレンダリングと処理機能を提供しています。この機能を最大限に活用するためには、LLMトレースを特定のフォーマットで記録する必要があります。

### input  
入力は、OpenAI互換形式のメッセージリストである必要があります。この形式はPythonの辞書（またはTypeScriptオブジェクト）として表現されます。
各メッセージは、次の2つのキーを含む必要があります：

- role : メッセージの役割（例: "user", "assistant", "system" など）
- content: メッセージの内容

### output  
以下のいずれかのフォーマットで出力を提供できます：
1.辞書/オブジェクト形式（choicesを含む）
- キー: choices
- 値: 辞書/オブジェクトのリスト
- 各辞書/オブジェクトは、以下の構造を持つ必要があります：
    - キー: message
    - 値: role と content を含むメッセージオブジェクト  
2.辞書/オブジェクト形式（messageを含む）  
- キー: message
- 値: role と content を含むメッセージオブジェクト  
3.タプル/配列形式  
2要素の配列またはタプル
- 1つ目の要素: role
- 2つ目の要素: content  
4.辞書/オブジェクト形式（role と content を含む）  

- 直接 role と content をキーとして持つ辞書/オブジェクト
入力名:

関数への入力は、messagesという名前で提供される必要があります。







### 追加のメタデータフィールド  
次のメタデータフィールドを提供すると、LangSmithがモデルを識別し、コストを計算するのに役立ちます。
LangChainやOpenAIのラッパーを使用する場合、これらのフィールドは自動的に正しく設定されます：

- ls_provider: モデルプロバイダーの名前（例: "openai", "anthropic" など）
- ls_model_name: モデル名（例: "gpt-4o-mini", "claude-3-opus-20240307" など）

In [6]:
from langsmith import traceable
from openai import OpenAI


inputs = [
    {"role": "system","content":"あなたは親切なアシスタントです"},
    {"role": "user", "content": "2人用の席を予約したい。"}
]

output = {
    "choices":[
        {
            "message": {
                "role":"assistant",
                "content":"喜んで。何時に予約したいですか？"
            }
        }
    ]
}

@traceable(run_type="llm",metadata={"is_provider":"human","is_models":"niikun"})
def chat_model(messages:list):
    return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': '喜んで。何時に予約したいですか？'}}]}

## Handling Streaming LLM Runs  

ストリーミングの場合、出力を「リデュース」して、非ストリーミングバージョンと同じ形式に統一することができます。この機能は現在、Pythonでのみサポートされています。

In [7]:
def _reduce_chunks(chunks:list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks]) 
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"},
    reduce_fn=_reduce_chunks
)
def my_streaming_chat_model(messages:list):
    for chunk in ["Hello, "+messages[1]["content"]]:
        yield {
            "choices": [
                {
                    "message": {
                        "role": "assistant",
                        "content": chunk
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system","content":"あなたは親切なアシスタントです。ユーザーに挨拶してください"},
            {"role": "user", "content": "2人用の席を予約したい。"}
        ]
    )
)

[{'choices': [{'message': {'role': 'assistant',
     'content': 'Hello, 2人用の席を予約したい。'}}]}]